# Nested Leave-One-Issue-Out Cross Validation For Random Forest On Textual Features using SMOTE

In [1]:
import numpy as np
import pandas as pd
import xlrd as xl
from pandas import ExcelWriter
from pandas import ExcelFile
import pprint
from gensim import corpora
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import gensim
import re
import pickle
from operator import itemgetter
import time, datetime
from functools import partial, update_wrapper
from beautifultable import BeautifulTable
from openpyxl import load_workbook

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN 

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as Imb_Pipeline

from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
# from imblearn.ensemble import BalancedBaggingClassifier

from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, LeaveOneGroupOut
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score, make_scorer, confusion_matrix

pp = pprint.PrettyPrinter(indent=4)

## Ignore warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
np.warnings.filterwarnings('ignore')

#### Use spaCy parser for word tokenization of a sentence:

In [2]:
import spacy
eng = spacy.load('en')
from spacy.lang.en import English
parser = English()

#### Define stopwords as punctuation + common contractions:

In [3]:
from string import punctuation
from nltk.corpus import stopwords

stop_words = list(punctuation) + ["'s","'m","n't","'re","-","'ll",'...'] #+ stopwords.words('english') 

#### Code to lemmatize and tokenize:

In [4]:
def get_lemma(item):
    return WordNetLemmatizer().lemmatize(item)

def tokenize(line):
    line_tokens = []
    tokens = parser(line)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            line_tokens.append('URL')
        elif token.orth_.startswith('@'):
            line_tokens.append('SCREEN_NAME')
        elif str(token) not in stop_words:
            line_tokens.append(get_lemma(token.lower_))
    return line_tokens

In [5]:
### Read from the pickled file
all_data = pd.read_pickle('../../data/text_conv_data.pkl')

print("Size of corpus: "+str(len(all_data)))

Size of corpus: 4330


In [6]:
X = all_data['Text Content'].values
y = all_data['Code'].values
doc_groups = all_data['Document'].values

print("Number of unique labels: "+str(len(set(y))))

labels = list(set(y))
labels.sort()

pp.pprint(labels)

Number of unique labels: 13
[   'Action on Issue',
    'Bug Reproduction',
    'Contribution and Commitment',
    'Expected Behaviour',
    'Investigation and Exploration',
    'Social Conversation',
    'Motivation',
    'Observed Bug Behaviour',
    'Potential New Issues and Requests',
    'Solution Discussion',
    'Task Progress',
    'Solution Usage',
    'Workarounds']


## Nested Cross-Validation on Random Forest:

In [7]:
# To be used within GridSearch
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# To be used in outer CV
logo = LeaveOneGroupOut()
outer_cv = logo.split(X,y,groups=doc_groups)

### Define Pipelines:
    
1. Pipeline2: tfidf + SMOTE

In [8]:
pipeline2 = Imb_Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('smote', SMOTE()),
    ('clf', RandomForestClassifier())
])

### Hyperparameters to search
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams and/or bigrams
    'clf__n_estimators': (10, 50, 100),
    'clf__min_samples_split': (2, 5, 10),
}

## Nested Cross Validation Using GridSearch

In [9]:
### Define and create the scoring functions

def score_func(y_true, y_pred, score_index, i):
    return(precision_recall_fscore_support(y_true,y_pred,labels=labels)[score_index][i])

def avg_score(y_true, y_pred, score_index):
    return precision_recall_fscore_support(y_true,y_pred,average='weighted')[score_index]

def sum_support(y_true, y_pred):
    return len(y_true)

### Create partials for each of the metrics returned
score_funcs = {v: partial(score_func, score_index=k) for k, v in {0:'precision',1:'recall',2:'fscore',3:'support'}.items()}
prec_score = partial(score_func, score_index=0)
update_wrapper(prec_score,score_func)
rec_score = partial(score_func, score_index=1)
update_wrapper(rec_score,score_func)
f_score = partial(score_func, score_index=2)
update_wrapper(f_score,score_func)
support_score = partial(score_func, score_index=3)
update_wrapper(support_score,score_func)

### Create a callable scoring function for each of the metrics for each classification label
scorer = {}
for label_id in range(0,13):
    scorer['label'+str(label_id)+'_precision'] = make_scorer(prec_score, i=label_id)
    scorer['label'+str(label_id)+'_recall'] = make_scorer(rec_score, i=label_id)
    scorer['label'+str(label_id)+'_fscore'] = make_scorer(f_score, i=label_id)
    scorer['label'+str(label_id)+'_support'] = make_scorer(support_score, i=label_id)

### Create a callable scoring function for avg/total of the metrics across classification labels
scorer['avg_precision'] = make_scorer(avg_score,score_index=0)
scorer['avg_recall'] = make_scorer(avg_score,score_index=1)
scorer['avg_fscore'] = make_scorer(avg_score,score_index=2)
scorer['total_support'] = make_scorer(sum_support)

### Perform Nested cross-validation on Pipeline2
outer_cv = logo.split(X,y,groups=doc_groups)

start = time.time()
clf2 = GridSearchCV(pipeline2, parameters, cv=inner_cv, scoring='f1_weighted')
clf2_results = cross_validate(clf2, X=X, y=y, cv=outer_cv, scoring=scorer)
print("Completed Pipeline2 scenario in "+ str(datetime.timedelta(seconds=(time.time()-start))))

Completed Pipeline2 scenario in 19:25:17.178837


# Display and Save Training and Testing Results for each Fold:

## Pipeline2 Results:

In [11]:
train_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])
test_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])

result_dict = {}

# writer = pd.ExcelWriter('../../results/LOIO.xlsx')
book = load_workbook('../../results/LOIO.xlsx')
writer = pd.ExcelWriter('../../results/LOIO.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

datalength = 0

for i in range(0,15):
    for label_id in range(0,13):
        train_report.loc[labels[label_id], :] = [clf2_results['train_label'+str(label_id)+'_precision'][i],clf2_results['train_label'+str(label_id)+'_recall'][i],clf2_results['train_label'+str(label_id)+'_fscore'][i],clf2_results['train_label'+str(label_id)+'_support'][i]]
        test_report.loc[labels[label_id], :] = [clf2_results['test_label'+str(label_id)+'_precision'][i],clf2_results['test_label'+str(label_id)+'_recall'][i],clf2_results['test_label'+str(label_id)+'_fscore'][i],clf2_results['test_label'+str(label_id)+'_support'][i]]

    train_report.loc['Avg/Total', :] = [clf2_results['train_avg_precision'][i],clf2_results['train_avg_recall'][i],clf2_results['train_avg_fscore'][i],clf2_results['train_total_support'][i]]
    test_report.loc['Avg/Total', :] = [clf2_results['test_avg_precision'][i],clf2_results['test_avg_recall'][i],clf2_results['test_avg_fscore'][i],clf2_results['test_total_support'][i]]

    fold_index = pd.DataFrame(data=[{'Fold':'Fold '+str(i)}])
    fold_index.to_excel(writer,'RTS',startrow=datalength, index=False)
    datalength+=(len(fold_index)+2)
    train_report.to_excel(writer,'RTS',startrow=datalength)
    datalength+=(len(train_report)+2)
    test_report.to_excel(writer,'RTS',startrow=datalength)
    datalength+=(len(test_report)+2)

    result_dict['RTS_train_'+str(i)] = train_report
    result_dict['RTS_test_'+str(i)] = test_report

    train_report = train_report.astype(float).round(2)
    test_report = test_report.astype(float).round(2)

    print("\n------------------------- FOLD "+str(i)+": -------------------------")
    print("\nTraining Results:")
    print(train_report)
    print("\nTest Results:")
    print(test_report)

writer.save()


------------------------- FOLD 0: -------------------------

Training Results:
                                   Precision  Recall  F1-score  Support
Action on Issue                         1.00    1.00      1.00      53.0
Bug Reproduction                        1.00    0.96      0.98     245.0
Contribution and Commitment             1.00    1.00      1.00      58.0
Expected Behaviour                      0.99    0.98      0.99     119.0
Investigation and Exploration           1.00    0.97      0.98     377.0
Social Conversation                     0.99    1.00      0.99     546.0
Motivation                              1.00    0.99      1.00     261.0
Observed Bug Behaviour                  0.96    0.95      0.96     131.0
Potential New Issues and Requests       1.00    1.00      1.00     208.0
Solution Discussion                     0.99    0.98      0.99    1338.0
Task Progress                           0.99    0.97      0.98     104.0
Solution Usage                          0.89 

### Get list of F1 scores and support-weighted average F1 score

In [12]:
avg_f1 = []
for i in range(0,15):
    avg_f1.append(float('%.4f' % round(clf2_results['test_avg_fscore'][i],4)))
    
print('Pipeline2:')
print(avg_f1)

sum = 0
den = 0
for i in range(0,15):
    sum += clf2_results['test_avg_fscore'][i] * clf2_results['test_total_support'][i]
    den += clf2_results['test_total_support'][i]
print('Pipeline2: Total weighted F-Score: '+('%.4f' % round(sum/den,4)))

Pipeline2:
[0.599, 0.2398, 0.155, 0.3303, 0.6295, 0.3266, 0.4834, 0.1869, 0.2323, 0.3524, 0.2003, 0.6424, 0.1942, 0.3894, 0.5082]
Pipeline2: Total weighted F-Score: 0.3629
